# How to Text Classification

Dieses Juypter Notebook wurde für den Vortrag *Textklassifikation mit Machine Learning* im Rahmen [DGPuK-Methoden-Tagung 2018 in Ilmenau](https://www.tu-ilmenau.de/en/computational-communication-science/dgpuk-methodentagung-2018/) erstellt und enthält ergänzenden Quellcode in der Programmiersprache Python (Python 3).

### Inhalt

* [Daten einlesen](#first-bullet)
* [Modell auswählen](#second-bullet)
* [Daten vorbereiten und aufteilen](#third-bullet)
* [Modell trainieren](#4-bullet)
* [Modell testen](#5-bullet)
* [Modell evaluieren](#6-bullet)


### Daten einlesen <a name="first-bullet"></a>

So kann man mit Hilfe der Bibliothek **Pandas** Daten aus einem .csv-File einlesen. Alternativ kann natürlich auch eine andere Bibliothek oder ein anderes Dateiformat gewählt werden.

In [64]:
import pandas as pd #Pandas-Bibliothek aufrufen (zukünftig unter dem Kürzel pd)

In [65]:
file = open("beispielposts.csv") #.csv-File aus einem lokale Speicherort einlesen.

In [66]:
df = pd.read_table(file, sep=';', encoding="utf-8") #File als pandas-DataFrame im Objekt df speichern.

In [67]:
df.head(15) #Zeigt die ersten 15 Fälle (Zeilen) des Data Frames plus Header.

,Text,Klasse
0,Der Grund ist der hohe Ausl?nderanteil. Die Ta...,1
1,Bis zu -24 Grad!!!! ?,1
2,Entsprechende Schritte w?rden in den kommenden...,1
3,"Die Hygiene-Center haben Toiletten, Duschen, W...",1
4,F?r die tollste Frau der Welt! <3,1
5,Mehr Platz f?r dich! :-*,1
6,"""Sie sagen, sch?rfere Waffengesetze verringern...",1
7,Ihr esst gerne Tiefk?hlpizza? Dann k?nnte es s...,1
8,Auf jeden Fall ges?ndere Besch?ftigung als Zoc...,1
9,Sonntagabend gemeinsam auf dem Sofa. ?? (via L...,1


Als X wird die Spalte Text definiert, als y die Spalte Klasse aus dem Datensatz `df`:

In [68]:
X = df.Text

len(X) #Gibt die Länge von X aus.

10000

In [69]:
y = df.Klasse

len(y) #Gibt die Länge von y aus.

10000

### Modell auswählen <a name="second-bullet"></a>

Als Beispiel benutzen wir einen *Naive Bayes Classifier* für multinominale Daten, der sich gut für Bag-of-Words Repräsentationen von Textdaten eignet. Der Classifier verfügt über die drei Parameter `alpha`, `class_prior` und `fit_prior`, die modifiziert werden können. Der Classifier wird zunächst in den Default-Einstellungen, dem Objekt `model` zugewiesen.

Im ML mit Python ist die populärste Bibliothek **scikit-learn**. Aus dieser Bibliothek importieren wir auch die Modelle für unsere Klassifikation.

In [70]:
from sklearn.naive_bayes import MultinomialNB #Lädt den Classifier MultinominalNB aus der sklearn-Bibliothek.


MultinomialNB()

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [71]:
model = MultinomialNB() #Weißt dem Objekt model den NB-Classifier zu.

### Daten vorbereiten und aufteilen <a name="third-bullet"></a>

Oft müssen die Daten noch mehr oder weniger aufwendig bearbeitet werden, bevor sie dem Modell zugeführt werden können. Für Textdaten kann diese Aufgabe besonders langwierig sein. Aufwendige Textverarbeitung ist am besten mit der Bibliothek **NLTK** umzusetzen. Aber auch die ML-Bibliothek scikit-learn kann einige (einfache) Vorverarbeitung und Transformation leisten. 

In jedem Fall werden aus der sklearn-Bibliothek die `Vectorizer` benötigt, um die Texte in eine Bag-of-Words Feature Matrix umzuwandeln. In dieser Form wird jedes Textdokument durch die absoluten oder gewichteten Worthäufigkeiten repräsentiert. Für eine absolute Häufigkeitszählung wird der `CountVectorizer` verwendet.

In [72]:
import sklearn #Lädt die Bibliothek scikit-learn.

In [73]:
#Importiert den CountVectorizer aus der sklearn-Bibliothek.
from sklearn.feature_extraction.text import CountVectorizer

Die `Vectorizer` in sklearn nehmen mehrere Argumente entgegen, die u.a. Informationen darüber enthalten, welche Wörter für die Vektorizierung berücksichtigt werden sollen.

In [74]:
#Die folgenden Argumente können diesem Vectorizer übergeben werden.
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [75]:
#Speichert den Vectorizer im Objekt vec.
vec = CountVectorizer()

Machen wir einen kurzen Text anhand eines Dokuments aus dem `df`. 

In [76]:
test = ["Vielen Dank für die Blumen, vielen Dank, wie lieb von dir."]

In [77]:
test_vector = vec.fit_transform(test)

In [78]:
#Der Feature-Vektor für den obigen Satz sieht nach der Vektorizierung wie folgt aus.
test_vector.toarray()

array([[1, 2, 1, 1, 1, 1, 2, 1, 1]], dtype=int64)

In [79]:
#Mit der Pandas-Bibliothek auch in schön:
pd.DataFrame(test_vector.toarray(),columns=vec.get_feature_names())

,blumen,dank,die,dir,für,lieb,vielen,von,wie
0,1,2,1,1,1,1,2,1,1


Nun müssen die Daten noch aufgeteilt werden. Im ML gibt es neben der folgenden Methode auch Verfahren, die den Datensatz mehrfach in Trainings- und Testsets aufteilen und stabiliere Schätzungen ermöglichen (-> Cross Validation).

In [80]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4321)

Die Vectorizer verfügen über die Methoden `fit` und `transform` (oder `fit_transform`, wenn man beides auf einmal ausführen möchte). An den Trainingsdaten wird der Vectorizer gefittet und die Texte transformiert. Im Testdatensatz fittet der Vectorizer nicht noch einmal an dem Vokabular, sondern transformiert nur auf Basis der Features, die er aus dem Trainset kennt.

In [81]:
X_train_vec = vec.fit_transform(X_train.astype("U"))
X_test_vec = vec.transform(X_test.astype("U"))

In [82]:
pd.DataFrame(X_train_vec.toarray(),columns=vec.get_feature_names())#feature Matrix des Trainsets.

,00,000,0007770,007,01,0180,02,030,032,04,...,zxzeof,zyedl0,zyhbor,zyklon,zyklop,zynismus,zypern,zyperns,zyprischen,zzhkra
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Modell trainieren <a name="4-bullet"></a>

Mit der `fit`-Methode, über die alle Classifier in sklearn verfügen, "lernt" das Modell, jedoch nur am Trainset, das aus `X_train_vec` und `y_train` besteht. 

Möchte man sich die Rechenzeit ausgeben lassen, geht das im Jupyter Notenbook über `%time`.

In [83]:
model.fit(X_train_vec, y_train) # %time model.fit(X_train_vect, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Modell testen <a name="5-bullet"></a>

Das trainierte Modell sagt nun die Klassen für die Testdaten voraus `X_test_vec`. Die Ergebnisse werden im Objekt `y_pred` gespeichert.

In [84]:
y_pred = model.predict(X_test_vec)

In [85]:
y_pred[:10] #Die ersten 10 vorhersagen sehen wie folgt aus:

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Modell evaluieren <a name="6-bullet"></a>

Für die Modellevaluation werden die wahren Werte für Y aus dem Testset `y_test` mit den geschätzten Werten `y_pred` durch das Modell `model` abgeglichen. Alles was wir brauchen finden wir in sklearn bei *metrics*.

In [86]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [87]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.801


In [88]:
pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    columns=['Predicted 0', 'Predicted 1'],
    index=['True 0', 'True 1']
)

,Predicted 0,Predicted 1
True 0,1562,32
True 1,366,40
